In [10]:
import os
import openai

# openai.api_key = "sk-xxx" #insert your API key here
openai.api_key = os.environ.get("OPENAI_API_KEY")   # API key is stored in .env file

In [11]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("stories.pdf") # load the pdf file
docs = loader.load()

In [12]:
docs[0].page_content

'Perspective A:\nJamie\'s surprise party was such a hit! We had decided on The Cozy Corner, that lovely local \nrestaurant with the brick walls and soft fairy lights. I got there around 6:45 PM to help set up and \nmake sure everything was in order. Jenny and Mark were already there, inflating balloons and \narranging them around the reserved table. We had ordered a chocolate cake with white frosting and \n“Happy 25th Jamie!” written on it in blue icing.\nThe moment Jamie walked in at 7:30 PM, we all shouted, "Surprise!" She looked genuinely \nshocked, her face turning a delightful shade of red. The funniest part of the evening was when \nJamie tried to blow out the candles and accidentally knocked over her glass of water. Thankfully, no\nharm was done, and we all had a good laugh.\nI think she loved the gifts, especially the hand-painted mug from Jenny. It was a heartwarming \nevening, and I\'m so glad I could be a part of it.\nPerspective B: \nI still can\'t believe we pulled off the

In [13]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

In [14]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain

llm = ChatOpenAI(temperature=0)

# Map
map_template = """You will be provided with unstructured data. The data contains one event two persons' descriptions with their perspectives. \ 
Your task is to analyze these perspectives from semantic point of view and list all differences between these two perspectives. Write consise summary for each perspectives.The following text are the perspectives of two persons. \:
{docs}
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [15]:
# Reduce
reduce_template = """The following text are summaries of the perspectives of two persons.:
"{text}"
LIST OF DIFFERENCES BETWEEN PERSPECTIVE_A AND PERSPECTIVE_B:"""

reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="text"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [16]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [17]:
print(map_reduce_chain.run(split_docs))

1. The name of the restaurant is different in each perspective: The Cozy Corner in Perspective A and The Cozy Nook in Perspective B.
2. The person in Perspective A arrived at 6:45 PM to help set up, while the person in Perspective B arrived just in time at 6:50 PM.
3. In Perspective A, Jenny and Mark were already at the restaurant inflating balloons and arranging them around the reserved table, while in Perspective B, Sarah, Jenny, and Mark were decorating and setting up balloons on the tables.
4. The description of the cake differs slightly in each perspective. In Perspective A, it is described as a chocolate cake with white frosting and "Happy 25th Jamie!" written in blue icing, while in Perspective B, it is described as a chocolate cake with "Happy 25th, Jamie!" written in sparkly blue letters.
5. The reaction of Jamie to the surprise is described differently in each perspective. In Perspective A, it is described as genuine and Jamie turned red, while in Perspective B, it is describ